In [1]:
## Set notebook to auto reload updated modules
%load_ext autoreload
%autoreload 2

In [15]:
import json
from pathlib import Path
import typing as t
import os
from dataclasses import dataclass, field

In [3]:
## Set notebook's working directory to project root
os.chdir("..")
display(f"CWD: {os.getcwd()}")

'CWD: /home/jack/git/cflarepy'

In [4]:
from cflarepy.libs import core_utils, setup
from cflarepy.libs.settings import CLOUDFLARE_SETTINGS

In [5]:
import pandas as pd
from cloudflare import Cloudflare

In [6]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [41]:
@dataclass
class CloudflareConfig(object):
    cf_api_email: str | None 
    cf_api_key: str | None = field(default=None, repr=False)
    cf_api_token: str | None = field(default=None, repr=False)
    
    @property
    def token(self):
        return self.cf_api_token
    
    @property
    def key(self):
        return self.cf_api_key
    
    @property
    def email(self):
        return self.cf_api_email


@dataclass
class NotebookConfig(object):
    cf_config: CloudflareConfig = None

In [42]:
def load_nb_config(config_file: str = "./notebooks/nb_config.json") -> NotebookConfig:
    if not config_file:
        raise ValueError("Missing path to a notebook config JSON file.")
    
    config_file: Path = Path(config_file).expanduser() if "~" in str(config_file) else Path(str(config_file)).absolute()
    
    if not config_file.exists():
        raise FileNotFoundError(f"Could not find notebook config file at '{config_file}'")
    
    display(f"Loading notebook config from: {config_file}")
    with open(config_file, "r") as f:
        nb_config_dict: dict = json.load(f)
    
    cf_config: CloudflareConfig = CloudflareConfig(**nb_config_dict["cloudflare"])
    nb_config: NotebookConfig = NotebookConfig(cf_config=cf_config)
    
    return nb_config

In [43]:
nb_config = load_nb_config()
if nb_config:
    display(nb_config)

'Loading notebook config from: /home/jack/git/cflarepy/notebooks/nb_config.json'

NotebookConfig(cf_config=CloudflareConfig(cf_api_email='jackenyon@gmail.com'))

In [5]:
def get_zones(cf_client: Cloudflare):
    zones = []

    display("Getting Cloudflare zones")
    try:
        for zone in cf_client.zones.list():
            zones.append(zone)
    except Exception as exc:
        msg = f"({type(exc)}) Error getting Cloudflare zones. Details: {exc}"
        display(f"[ERROR] {msg}")
        
        raise exc
        
    return zones

In [9]:
api_token = CLOUDFLARE_SETTINGS.get("CF_API_TOKEN")
if not api_token:
    raise ValueError("Missing Cloudflare API token. Set environment variable CF_API_TOKEN=<API_TOKEN>")

display(api_token)


ValueError: Missing Cloudflare API token. Set environment variable CF_API_TOKEN=<API_TOKEN>

In [ ]:
cf_client =  Cloudflare(api_token=settings.CLOUDFLARE_SETTINGS.get("CF_API_TOKEN"))

In [ ]:
zones = get_zones(cf_client=cf_client)
zones